# zilliz cloud setup

In [17]:
from pymilvus import MilvusClient
import yaml
from pathlib import Path

# Load Zilliz secrets from config/secrets.yaml
# In notebooks, __file__ is not defined. Use the current working directory as base.
secrets_path = Path.cwd().parent / "config" / "secrets.yaml"
with open(secrets_path, "r", encoding="utf-8") as f:
    secrets = yaml.safe_load(f)
zilliz_conf = secrets["zilliz"]

# Prepare connection parameters
uri = zilliz_conf["endpoint"]
user = zilliz_conf["user"]
password = zilliz_conf["password"]
collection_name = zilliz_conf["collection_name"]

# MilvusClient expects "http(s)://host:port" and "user:password" as token
client = MilvusClient(
    uri=uri,
    token=f"{user}:{password}",
    secure=True
)
client


In [13]:

# Example: alter a field's max_length (e.g., "name" to 1024)
field_name = "name"
field_params = {"max_length": 200}

# This will update the field's max_length in the collection schema
client.alter_collection_field(
    collection_name=collection_name,
    field_name=field_name,
    field_params=field_params,
    nullable=True
)

print(f"Altered field '{field_name}' in collection '{collection_name}' to {field_params}")


Altered field 'name' in collection 'CN_candidates' to {'max_length': 200}


In [ ]:
from pymilvus import MilvusClient, DataType, FieldSchema
client.add_collection_field(
    collection_name=collection_name,
    field_name='stage',  # Name of the new field to add
    data_type=DataType.VARCHAR,
    nullable=True,
    max_length=20,
)

#2025-10-08 19:03:56,737 [ERROR][handler]: grpc RpcError: [add_collection_field], <_InactiveRpcError: StatusCode.UNIMPLEMENTED, unknown method AddCollectionField for service milvus.proto.milvus.MilvusService>, <Time:{'RPC start': '2025-10-08 19:03:56.545375', 'gRPC error': '2025-10-08 19:03:56.737714'}> (decorators.py:151)


In [19]:
from datetime import datetime
import random
import string

# Generate a fake candidate record
fake_record = {
    # "candidate_id": ''.join(random.choices(string.ascii_lowercase + string.digits, k=16)),
    "chat_id": ''.join(random.choices(string.ascii_lowercase + string.digits, k=12)),
    "name": "张三",
    "job_applied": "AI算法工程师",
    "resume_text": "本人有丰富的AI算法开发经验，熟悉Python、深度学习框架，曾在知名互联网公司任职。",
    "resume_vector": [random.uniform(-1, 1) for _ in range(1536)],
    "stage": "GREET",
    "metadata": {
        "thread_id": ''.join(random.choices(string.ascii_lowercase + string.digits, k=24)),
        "assistant_id": ''.join(random.choices(string.ascii_lowercase + string.digits, k=24)),
        "created_at": datetime.now().isoformat(),
    },
    "updated_at": datetime.now().isoformat(),
}

# Insert the fake record into the collection
insert_result = client.insert(
    collection_name=collection_name,
    data=[fake_record]
)
print("Inserted fake record:", insert_result)


2025-10-09 12:31:04,397 [ERROR][handler]: RPC error: [insert_rows], <DataNotMatchException: (code=1, message=Insert missed an field `candidate_id` to collection without set nullable==true or set default_value)>, <Time:{'RPC start': '2025-10-09 12:31:04.333440', 'RPC error': '2025-10-09 12:31:04.397583'}> (decorators.py:140)


DataNotMatchException: <DataNotMatchException: (code=1, message=Insert missed an field `candidate_id` to collection without set nullable==true or set default_value)>